<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Market クラス解説

Marketクラスは、取引所で扱われる通貨ペアーの１組合わせをモデリングしたクラスです。
現在はBinanceの USDBUSDのSPOT取引しか扱えませんが、比較的容易に拡張できるようにつくっていますので順次拡張していきます。

## Marketクラスメソッド

### マーケットオブジェクトの生成(`open`)

`open`メソッドでMaketオブジェクトを開きます。内部的にキャッシュされているので２回目からはキャッシュを返します。そのため複数回呼び出してOKです。

今後SPOT以外にも対応した場合、パラメータが変更になる可能性があります。
```
binance = Market.open(
    'BN',               # Binanceは 'BN'
    'BTCBUSD'
)
```

### データのダウンロード(`download`)

指定した日付あらあとのデータをダウンロードしてDBへ保存します。
BTCBUSDしか対応していないため効果はありませんが、`open`で開いたマーケットオブジェクト全てに対してダウンロード処理をします。

```
Market.download(
    5   #ダウンロード日数（N日前を指定)
    )
```


## Marketオブジェクトメソッド

Marketインスタンスが`binance`オブジェクトに保存されている前提で説明します。

###　情報表示　

####　`_repr_html_()`

jupyuter note上でオブジェクトを入力してリターンすると内部的に`_repr_html_()`がよばれて内容が分かりやすく表示されます。
```
binance
```

#### DBのパス表示 `file_path`

DBは巨大になりますので、不要になったら`file_path`プロパティで表示される場所を確認して手動で消してください。

```
binance.file_name
```

### OHLCV取得（pd.DataFrame形式)

時刻はμsで指定する。開始時刻に0を指定するとDBの最初から、終了に0を指定するとDBの最後まで（現在時刻に一番ちかいときまで）を指定したことになる。

```
binance.ohlcv(
    start_timestamp_us,     #開始時刻（μs）
    end_timestamp_us,       #終了時刻（μs)
    60                      #足の幅（秒）
    )
```

pd.DataFrame 形式でOHLCVデータが取得できます。この機能だけつかっても便利に使えます。
なお、Tickデータから任意の足を再生成しているので初回キャッシュされていないときはかなり時間がかかります。




## 以下動かしながら説明

### Google Collab向けパッケージインストール

In [1]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh > 3

# https://github.com/yasstake/rusty-bot/releases/tag/release-0.2.0
# （環境にあわせて修正お願いします）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 6.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: panel 0.12.1
Uninstalling panel-0.12.1:
  Successfully uninstalled panel-0.12.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 24.6 MB/s eta 0:00:00


### 以後Marktライブラリの動作

In [2]:
# 必要ライブラリのimport
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [4]:
# Binance('BN')の'BTCBUSD'ペアーの取引マーケットを生成
binance = Market.open('BN', 'BTCBUSD')

# 上記でオープンしたマーケットに対し5日より新しいログをダウンロード
Market.download(5)

# Market.download(5, True)       #ディフォルトではキャッシュされているので、再ダウンロードしたい場合は強制フラグをTrueにする。Collabでは１日あたり1分ぐらいかかります。

In [5]:
# Marketオブジェクトが管理するDBに保存されているデータ期間を表示する。
binance

start,end
1670889600100000,1671407999900000
"""2022-12-13T00:00:00.100000""","""2022-12-18T23:59:59.900000"""
days=,5


In [6]:
# データベースのDisk上の場所を表示する。不要になったら手で消してください。

binance.file_name

'/root/.local/share/rbot/DB/BN/SPOT-BTCBUSD.db'

## OHLCVの取得

In [7]:
# 初回は遅いです。２回目はキャッシュされているので比較的高速です
binance.ohlcv(
    DAYS_BEFORE(5),     # 5日前から
    0,                  # 最新まで
    60*60
)

,open,high,low,close,volume
timestamp,,,,,
2022-12-14 10:00:00+00:00,17827.18,17830.00,17811.82,17828.15,993.65398
2022-12-14 11:00:00+00:00,17828.15,17859.00,17807.00,17825.88,3778.80179
2022-12-14 12:00:00+00:00,17824.90,17930.00,17815.21,17914.61,6847.23861
2022-12-14 13:00:00+00:00,17914.70,17964.82,17862.80,17895.76,6236.92182
2022-12-14 14:00:00+00:00,17895.41,18094.00,17894.13,18062.34,12078.83386
...,...,...,...,...,...
2022-12-18 19:00:00+00:00,16748.27,16771.86,16725.01,16753.31,2477.76072
2022-12-18 20:00:00+00:00,16753.20,16774.87,16749.09,16759.96,2279.32264
2022-12-18 21:00:00+00:00,16759.68,16785.00,16738.11,16749.24,2285.04540


短めの足では任意のサイズの足が作れるので、このOHCLVだけを利用して独自の分析を行うにも便利だと思います。
週足より長い足も作れますが、性能が出ないのとデータが揃わない点に問題があり実用できではないかもしれません。

ご活用ください。